## Análise Fatorial e PCA
### MBA em Data Science e Analytics USP ESALQ

**Prof Dr.** Wilson Tarantin Junior

**Aluna:** Luiza Batista Laquini

**Turma:** DSA 2024-1

### Bibliotecas e configurações

In [ ]:
import pandas as pd
import numpy as np
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns

#import plotly.io as pio
#pio.renderers.default='browser'

import warnings
warnings.filterwarnings("ignore")

% Importando o banco de dados

### Visualização dos dados

In [ ]:
emprestimo = pd.read_excel("emprestimo_banco.xlsx")
# Fonte: adaptado de https://www.kaggle.com/datasets/itsmesunil/bank-loan-modelling

% Separando somente as variÃ¡veis para a anÃ¡lise fatorial PCA

In [ ]:
emprestimo_pca = emprestimo.drop(columns=['ID'])

% EstatÃ­sticas descritivas das variÃ¡veis

In [ ]:
tab_desc = emprestimo_pca.describe()

% Matriz de correlaÃ§Ãµes de Pearson

Matriz de correlaÃ§Ãµes em um objeto "simples"

In [ ]:
corr = emprestimo_pca.corr()

GrÃ¡fico interativo

In [ ]:
fig = go.Figure()

In [ ]:
fig.add_trace(
    go.Heatmap(
        x = corr.columns,
        y = corr.index,
        z = np.array(corr),
        text=corr.values,
        texttemplate='%{text:.3f}',
        colorscale='viridis'))

In [ ]:
fig.update_layout(
    height = 600,
    width = 600,
    yaxis=dict(autorange="reversed"))

In [ ]:
fig.show()

% Teste de Esfericidade de Bartlett

In [ ]:
bartlett, p_value = calculate_bartlett_sphericity(emprestimo_pca)

In [ ]:
print(f'QuiÂ² Bartlett: {round(bartlett, 2)}')
print(f'p-valor: {round(p_value, 4)}')

% Definindo a PCA (procedimento inicial com todos os fatores possÃ­veis)

In [ ]:
fa = FactorAnalyzer(n_factors=6, method='principal', rotation=None).fit(emprestimo_pca)

% Obtendo os eigenvalues (autovalores): resultantes da funÃ§Ã£o FactorAnalyzer

In [ ]:
autovalores = fa.get_eigenvalues()[0]

In [ ]:
print(autovalores)

% Redefinindo a PCA pelo critÃ©rio da raiz latente

HÃ¡ dois fatores derivados de autovalores > 1

In [ ]:
fa = FactorAnalyzer(n_factors=2, method='principal', rotation=None).fit(emprestimo_pca)

% Eigenvalues, variÃ¢ncias e variÃ¢ncias acumuladas

In [ ]:
autovalores_fatores = fa.get_factor_variance()

In [ ]:
tabela_eigen = pd.DataFrame(autovalores_fatores)
tabela_eigen.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_eigen.columns)]
tabela_eigen.index = ['Autovalor','VariÃ¢ncia', 'VariÃ¢ncia Acumulada']
tabela_eigen = tabela_eigen.T

In [ ]:
print(tabela_eigen)

% Determinando as cargas fatoriais

In [ ]:
cargas_fatoriais = fa.loadings_

In [ ]:
tabela_cargas = pd.DataFrame(cargas_fatoriais)
tabela_cargas.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_cargas.columns)]
tabela_cargas.index = emprestimo_pca.columns

In [ ]:
print(tabela_cargas)

% GrÃ¡fico das cargas fatoriais (loading plot)

In [ ]:
plt.figure(figsize=(12,8))
tabela_cargas_chart = tabela_cargas.reset_index()
plt.scatter(tabela_cargas_chart['Fator 1'], tabela_cargas_chart['Fator 2'], s=30, color='red')

In [ ]:
def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x'] + 0.02, point['y'], point['val'], fontsize=8)

In [ ]:
label_point(x = tabela_cargas_chart['Fator 1'],
            y = tabela_cargas_chart['Fator 2'],
            val = tabela_cargas_chart['index'],
            ax = plt.gca()) 

In [ ]:
plt.axhline(y=0, color='grey', ls='--')
plt.axvline(x=0, color='grey', ls='--')
plt.ylim([-1.1,1.1])
plt.xlim([-1.1,1.1])
plt.title("Loading Plot", fontsize=16)
plt.xlabel(f"Fator 1: {round(tabela_eigen.iloc[0]['VariÃ¢ncia']*100,2)}% de variÃ¢ncia explicada", fontsize=12)
plt.ylabel(f"Fator 2: {round(tabela_eigen.iloc[1]['VariÃ¢ncia']*100,2)}% de variÃ¢ncia explicada", fontsize=12)
plt.show()

% Determinando as comunalidades

In [ ]:
comunalidades = fa.get_communalities()

In [ ]:
tabela_comunalidades = pd.DataFrame(comunalidades)
tabela_comunalidades.columns = ['Comunalidades']
tabela_comunalidades.index = emprestimo_pca.columns

In [ ]:
print(tabela_comunalidades)

% ExtraÃ§Ã£o dos fatores para as observaÃ§Ãµes do banco de dados

In [ ]:
fatores = pd.DataFrame(fa.transform(emprestimo_pca))
fatores.columns =  [f"Fator {i+1}" for i, v in enumerate(fatores.columns)]

% Identificando os scores fatoriais

In [ ]:
scores = fa.weights_

In [ ]:
tabela_scores = pd.DataFrame(scores)
tabela_scores.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_scores.columns)]
tabela_scores.index = emprestimo_pca.columns

In [ ]:
print(tabela_scores)

% Em certos casos, a "rotaÃ§Ã£o de fatores" pode melhorar a interpretaÃ§Ã£o

Analisando pelo loading plot, aplica-se a rotaÃ§Ã£o dos eixos na origem (0,0)<br>
O mÃ©todo mais comum Ã© a 'varimax', que Ã© a rotaÃ§Ã£o ortogonal dos fatores<br>
O objetivo Ã© aumentar a carga fatorial em um fator e diminuir em outro<br>
Em resumo, trata-se de uma redistribuiÃ§Ã£o de cargas fatoriais

% Adicionando a rotaÃ§Ã£o: rotation='varimax'

Aplicando a rotaÃ§Ã£o aos 2 fatores extraÃ­dos

In [ ]:
fa = FactorAnalyzer(n_factors=2, method='principal', rotation='varimax').fit(emprestimo_pca)

% Eigenvalues, variÃ¢ncias e variÃ¢ncias acumuladas

In [ ]:
autovalores_fatores = fa.get_factor_variance()

In [ ]:
tabela_eigen = pd.DataFrame(autovalores_fatores)
tabela_eigen.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_eigen.columns)]
tabela_eigen.index = ['Autovalor','VariÃ¢ncia', 'VariÃ¢ncia Acumulada']
tabela_eigen = tabela_eigen.T

In [ ]:
print(tabela_eigen)

 HÃ¡ a redistribuiÃ§Ã£o da variÃ¢ncia entre os fatores (mas o total Ã© o mesmo!)

% Determinando as cargas fatoriais

In [ ]:
cargas_fatoriais = fa.loadings_

In [ ]:
tabela_cargas = pd.DataFrame(cargas_fatoriais)
tabela_cargas.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_cargas.columns)]
tabela_cargas.index = emprestimo_pca.columns

In [ ]:
print(tabela_cargas)

 As cargas sÃ£o alteradas: cargas fatoriais rotacionadas

% GrÃ¡fico das cargas fatoriais (loading plot)

In [ ]:
plt.figure(figsize=(12,8))
tabela_cargas_chart = tabela_cargas.reset_index()
plt.scatter(tabela_cargas_chart['Fator 1'], tabela_cargas_chart['Fator 2'], s=30, color='blue')

In [ ]:
def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x'] + 0.02, point['y'], point['val'], fontsize=8)

In [ ]:
label_point(x = tabela_cargas_chart['Fator 1'],
            y = tabela_cargas_chart['Fator 2'],
            val = tabela_cargas_chart['index'],
            ax = plt.gca()) 

In [ ]:
plt.axhline(y=0, color='grey', ls='--')
plt.axvline(x=0, color='grey', ls='--')
plt.ylim([-1.1,1.1])
plt.xlim([-1.1,1.1])
plt.title("Loading Plot - Fatores Rotacionados", fontsize=16)
plt.xlabel(f"Fator 1: {round(tabela_eigen.iloc[0]['VariÃ¢ncia']*100,2)}% de variÃ¢ncia explicada", fontsize=12)
plt.ylabel(f"Fator 2: {round(tabela_eigen.iloc[1]['VariÃ¢ncia']*100,2)}% de variÃ¢ncia explicada", fontsize=12)
plt.show()

% Determinando as comunalidades

In [ ]:
comunalidades = fa.get_communalities()

In [ ]:
tabela_comunalidades = pd.DataFrame(comunalidades)
tabela_comunalidades.columns = ['Comunalidades']
tabela_comunalidades.index = emprestimo_pca.columns

In [ ]:
print(tabela_comunalidades)

 As comunalidades nÃ£o mudam!

% ExtraÃ§Ã£o dos fatores para as observaÃ§Ãµes do banco de dados

In [ ]:
fatores = pd.DataFrame(fa.transform(emprestimo_pca))
fatores.columns =  [f"Fator {i+1}" for i, v in enumerate(fatores.columns)]

Adicionando os fatores ao banco de dados

In [ ]:
emprestimo = pd.concat([emprestimo.reset_index(drop=True), fatores], axis=1)

% Identificando os scores fatoriais

In [ ]:
scores = fa.weights_

In [ ]:
tabela_scores = pd.DataFrame(scores)
tabela_scores.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_scores.columns)]
tabela_scores.index = emprestimo_pca.columns

In [ ]:
print(tabela_scores)

 Os scores sÃ£o alterados, o que geram fatores rotacionados!

% Interpretando os scores fatoriais em cada fator extraÃ­do

In [ ]:
tabela_scores_graph = tabela_scores.reset_index()
tabela_scores_graph = tabela_scores_graph.melt(id_vars='index')

In [ ]:
sns.barplot(data=tabela_scores_graph, x='variable', y='value', hue='index', palette='viridis')
plt.legend(title='VariÃ¡veis', bbox_to_anchor=(1,1), fontsize = '6')
plt.title('Scores Fatoriais', fontsize='12')
plt.xlabel(xlabel=None)
plt.ylabel(ylabel=None)
plt.show()

% Os fatores continuam ortogonais

In [ ]:
pg.rcorr(emprestimo[['Fator 1', 'Fator 2']], 
         method = 'pearson', upper = 'pval', 
         decimals = 4, 
         pval_stars = {0.01: '***', 0.05: '**', 0.10: '*'})

% Fim!